## Import File Yang Dibutuhkan

In [1]:
from google.colab import files
uploaded = files.upload()

Saving stki-uas-A11.2023.14979.zip to stki-uas-A11.2023.14979.zip


In [2]:
import zipfile
import os

with zipfile.ZipFile("stki-uas-A11.2023.14979.zip", 'r') as zip_ref:
    zip_ref.extractall("project")

os.listdir("project")

['stki-uas-A11.2023.14979']

# UAS Sistem Temu Kembali Informasi

Nama : Suluh Yoga Pratama  
NIM  : A11.2023.14979  

Notebook ini berisi implementasi dan pengujian:
- Preprocessing
- Boolean Information Retrieval
- Vector Space Model (TF-IDF)
- Clustering Dokumen
- Text Summarization

## 1. Set Directory & Setup Library

In [3]:
import os
os.chdir("/content/project/stki-uas-A11.2023.14979")

In [4]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 133.4 MB/s eta 0:00:00


In [5]:
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## 2. Import Modul Project

In [7]:
import sys
sys.path.append(os.getcwd())

In [8]:
from src.preprocess import preprocess_text
from src.search import search_boolean, search_vector
from src.clustering import clustering_kmeans
from src.summarization import summarize_document

## 3. Preprocessing

In [11]:
sample_text = """
Ekonomi digital berkembang pesat di Indonesia.
Transformasi teknologi mendorong pertumbuhan UMKM nasional.
"""

tokens = preprocess_text(sample_text)

print("HASIL PREPROCESSING")
print("-"*40)
print("Jumlah token:", len(tokens))
print("Token (contoh):")
print(", ".join(tokens[:15]))

HASIL PREPROCESSING
----------------------------------------
Jumlah token: 11
Token (contoh):
ekonomi, digital, kembang, pesat, indonesia, transformasi, teknologi, dorong, tumbuh, umkm, nasional


## 4. Boolean IR

In [12]:
queries = [
    "ekonomi",
    "ekonomi AND digital",
    "hukum OR sosial",
    "NOT kesehatan"
]

for q in queries:
    print("\nQUERY:", q)
    print("-"*40)

    results = search_boolean(q)

    if not results:
        print("Tidak ada dokumen ditemukan.")
    else:
        for r in results[:10]:
            print("•", r)


QUERY: ekonomi
----------------------------------------
• ekonomi_bank_mandiri_prediksi_ekonomi_indonesia_tumbuh_44__003.txt
• ekonomi_di_forum_internasional_sri_mulyani_cerita_ekonomi__009.txt
• ekonomi_ekonomi_bakal_pulih_usai_penemuan_vaksin_covid19_b_008.txt
• ekonomi_ekonomi_ri_bakal_membaik_intip_3_sektor_saham_pili_004.txt
• ekonomi_gubernur_bi_perry_warjiyo_ungkap_3_cara_akselerasi_006.txt
• ekonomi_jokowi_komitmen_lanjutkan_program_bansos_dan_keseh_007.txt
• ekonomi_menko_airlangga_yakin_ekonomi_indonesia_tumbuh_55__001.txt
• ekonomi_pelabuhan_patimban_bakal_tingkatkan_kesejahteraan__010.txt
• ekonomi_realisasi_investasi_asing_bakal_bantu_pertumbuhan__002.txt
• ekonomi_vaksinasi_covid19_jadi_salah_satu_strategi_pemerin_005.txt

QUERY: ekonomi AND digital
----------------------------------------
• ekonomi_bank_mandiri_prediksi_ekonomi_indonesia_tumbuh_44__003.txt
• ekonomi_realisasi_investasi_asing_bakal_bantu_pertumbuhan__002.txt
• ekonomi_vaksinasi_covid19_jadi_salah_satu_st

## 5. Vector Space Model (VSM)

In [13]:
query = "ekonomi teknologi digital"

results = search_vector(query)

print("HASIL VECTOR SPACE MODEL")
print("-"*40)

for i, (doc, score) in enumerate(results, start=1):
    print(f"{i}. {doc}")
    print(f"   Skor Similaritas: {score:.4f}")

HASIL VECTOR SPACE MODEL
----------------------------------------
1. teknologi_ketika_syariat_islam_berpadu_dengan_teknologi_016.txt
   Skor Similaritas: 0.2591
2. ekonomi_vaksinasi_covid19_jadi_salah_satu_strategi_pemerin_005.txt
   Skor Similaritas: 0.2520
3. teknologi_lewat_teknologi_pasarpolis_manfaatkan_peluang_gap__015.txt
   Skor Similaritas: 0.2292
4. ekonomi_realisasi_investasi_asing_bakal_bantu_pertumbuhan__002.txt
   Skor Similaritas: 0.1916
5. ekonomi_jokowi_komitmen_lanjutkan_program_bansos_dan_keseh_007.txt
   Skor Similaritas: 0.1894
6. ekonomi_di_forum_internasional_sri_mulyani_cerita_ekonomi__009.txt
   Skor Similaritas: 0.1749
7. sosial_1387_hoaks_beredar_di_media_sosial_hngga_26_januar_036.txt
   Skor Similaritas: 0.1723
8. ekonomi_bank_mandiri_prediksi_ekonomi_indonesia_tumbuh_44__003.txt
   Skor Similaritas: 0.1710
9. ekonomi_gubernur_bi_perry_warjiyo_ungkap_3_cara_akselerasi_006.txt
   Skor Similaritas: 0.1682
10. ekonomi_ekonomi_bakal_pulih_usai_penemuan_vaksin_c

## 6. Clustering

In [14]:
clusters = clustering_kmeans("data/processed", k=5)

print("HASIL CLUSTERING")
print("="*50)

for label in sorted(clusters.keys()):
    docs = clusters[label]
    print(f"\nCluster {label} ({len(docs)} dokumen)")
    print("-"*40)
    for d in docs:
        print("•", d)

HASIL CLUSTERING

Cluster 0 (7 dokumen)
----------------------------------------
• hukum_pakar_hukum_sebut_penyitaan_aset_terkait_tppu_jang_024.txt
• hukum_pakar_penegak_hukum_perlu_hatihati_sita_aset_terka_028.txt
• ekonomi_ekonomi_ri_bakal_membaik_intip_3_sektor_saham_pili_004.txt
• sosial_harga_telur_ayam_anjlok_parah_peternak_salahkan_pe_033.txt
• teknologi_ride_vision_kembangkan_keselamatan_sepeda_motor_018.txt
• hukum_instagram_sammy_simorangkir_diretas_istri_sigap_am_026.txt
• ekonomi_gubernur_bi_perry_warjiyo_ungkap_3_cara_akselerasi_006.txt

Cluster 1 (7 dokumen)
----------------------------------------
• sosial_ikut_challenge_di_media_sosial_sambil_jaga_kesehat_032.txt
• kesehatan_ppkm_bisa_berjalan_efektif_bila_masyarakat_patuh_p_049.txt
• hukum_kasus_siswi_nonmuslim_disuruh_berjilbab_kuasa_huku_025.txt
• sosial_belasan_pelanggar_tertib_masker_di_cilandak_disank_038.txt
• teknologi_4_tren_teknologi_jaringan_menurut_aruba_pada_2021_013.txt
• sosial_faktor_sosial_dan_struktura

## 7. Text Summarization

In [15]:
import os

raw_folder = "data/raw"
files = [f for f in os.listdir(raw_folder) if f.endswith(".txt")]

file_path = os.path.join(raw_folder, files[0])

with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

summary = summarize_document(raw_text)

print("RINGKASAN DOKUMEN")
print("="*50)
print("Dokumen:", files[0])
print("-"*40)
print(summary)

RINGKASAN DOKUMEN
Dokumen: sosial_1387_hoaks_beredar_di_media_sosial_hngga_26_januar_036.txt
----------------------------------------
📌 Judul:
1.387 Hoaks Beredar di Media Sosial Hngga 26 Januari 2021

📝 Ringkasan Artikel:
Kominfo pun menandainya sebagai hoaks," tutur Semuel. Hoaks sebanyak itu tercatat sejak Maret 2020 hingga 26 Januari 2021.


## 8. Evaluasi Information Retrieval (Precision & Recall)

In [17]:
from src.search import search_boolean, search_vector
from src.eval import precision_recall

query = "ekonomi"

# Ground truth manual (berdasarkan kategori dataset)
gold_relevant = {
    "ekonomi_bank_mandiri_prediksi_ekonomi_indonesia_tumbuh_44__003.txt",
    "ekonomi_di_forum_internasional_sri_mulyani_cerita_ekonomi__009.txt",
    "ekonomi_ekonomi_bakal_pulih_usai_penemuan_vaksin_covid19_b_008.txt"
}

print("EVALUASI INFORMATION RETRIEVAL")
print("="*60)
print("Query:", query)
print("Ground Truth:", gold_relevant)

# Boolean IR
boolean_result = search_boolean(query)

p, r, tp, fp, fn = precision_recall(boolean_result, gold_relevant)

print("\n--- Boolean IR ---")
print(f"Precision : {p:.3f}")
print(f"Recall    : {r:.3f}")
print(f"TP={tp}, FP={fp}, FN={fn}")

# Vector Space Model
vsm_result = [doc for doc, _ in search_vector(query)]

p, r, tp, fp, fn = precision_recall(vsm_result, gold_relevant)

print("\n--- Vector Space Model ---")
print(f"Precision : {p:.3f}")
print(f"Recall    : {r:.3f}")
print(f"TP={tp}, FP={fp}, FN={fn}")

EVALUASI INFORMATION RETRIEVAL
Query: ekonomi
Ground Truth: {'ekonomi_ekonomi_bakal_pulih_usai_penemuan_vaksin_covid19_b_008.txt', 'ekonomi_di_forum_internasional_sri_mulyani_cerita_ekonomi__009.txt', 'ekonomi_bank_mandiri_prediksi_ekonomi_indonesia_tumbuh_44__003.txt'}

--- Boolean IR ---
Precision : 0.188
Recall    : 1.000
TP=3, FP=13, FN=0

--- Vector Space Model ---
Precision : 0.300
Recall    : 1.000
TP=3, FP=7, FN=0


## 9. Evaluasi Text Summarization (ROUGE-1)

In [18]:
from nltk.tokenize import word_tokenize
from src.summarization import summarize_text

def rouge_1(system_summary, reference_summary):
    sys_tokens = set(word_tokenize(system_summary.lower()))
    ref_tokens = set(word_tokenize(reference_summary.lower()))

    overlap = sys_tokens & ref_tokens
    precision = len(overlap) / len(sys_tokens) if sys_tokens else 0
    recall = len(overlap) / len(ref_tokens) if ref_tokens else 0

    return precision, recall

text = """
Ekonomi digital berkembang pesat di Indonesia.
Transformasi teknologi mendorong pertumbuhan UMKM.
Namun, tantangan regulasi masih menjadi kendala utama.
"""

reference_summary = "Ekonomi digital berkembang pesat dan mendorong UMKM."

system_summary = summarize_text(text, max_sentences=1)

p, r = rouge_1(system_summary, reference_summary)

print("EVALUASI TEXT SUMMARIZATION (ROUGE-1)")
print("="*60)
print("System Summary   :", system_summary)
print("Reference Summary:", reference_summary)
print(f"Precision : {p:.3f}")
print(f"Recall    : {r:.3f}")

EVALUASI TEXT SUMMARIZATION (ROUGE-1)
System Summary   : 
Ekonomi digital berkembang pesat di Indonesia.
Reference Summary: Ekonomi digital berkembang pesat dan mendorong UMKM.
Precision : 0.714
Recall    : 0.625


## 10. Analisis, Diskusi, dan Kesimpulan

# <b>Evaluasi Information Retrieval</b>

Pengujian sistem temu kembali informasi dilakukan menggunakan satu query uji, yaitu "ekonomi", dengan tiga dokumen relevan yang ditentukan secara manual sebagai ground truth.

# <b>Hasil evaluasi Boolean Information Retrieval menunjukkan:</b>

Precision = 0.188

Recall = 1.000

True Positive (TP) = 3

False Positive (FP) = 13

False Negative (FN) = 0

Nilai recall sebesar 1.000 menunjukkan bahwa seluruh dokumen relevan berhasil ditemukan oleh sistem. Namun, nilai precision yang rendah mengindikasikan bahwa sistem juga mengambil cukup banyak dokumen yang tidak relevan. Hal ini sesuai dengan karakteristik model Boolean yang hanya mempertimbangkan keberadaan kata kunci tanpa memperhitungkan tingkat relevansi dokumen.

# <b>Hasil evaluasi Vector Space Model (TF-IDF) menunjukkan:</b>

Precision = 0.300

Recall = 1.000

True Positive (TP) = 3

False Positive (FP) = 7

False Negative (FN) = 0

Model Vector Space Model mampu mempertahankan nilai recall maksimal, sekaligus meningkatkan nilai precision dibandingkan Boolean IR. Hal ini menunjukkan bahwa pendekatan berbasis pembobotan TF-IDF dan cosine similarity lebih efektif dalam menyaring dokumen yang relevan.

# <b>Evaluasi Text Summarization</b>

Evaluasi peringkasan teks dilakukan menggunakan metrik ROUGE-1 dengan membandingkan ringkasan sistem terhadap ringkasan referensi.

Hasil evaluasi menunjukkan:

Precision = 0.714

Recall = 0.625

Nilai precision yang cukup tinggi menunjukkan bahwa sebagian besar kata pada ringkasan sistem juga terdapat pada ringkasan referensi. Sementara itu, nilai recall yang lebih rendah mengindikasikan bahwa masih terdapat informasi penting dalam ringkasan referensi yang belum sepenuhnya tertangkap oleh sistem.

# <b>Kesimpulan</b>

Berdasarkan hasil evaluasi, sistem temu kembali informasi yang dibangun telah berfungsi dengan baik. Model Vector Space Model (TF-IDF) menunjukkan performa yang lebih baik dibandingkan Boolean Information Retrieval dalam hal precision, dengan tetap mempertahankan recall maksimal.

Pada fitur peringkasan teks, metode ekstraktif berbasis frekuensi kata mampu menghasilkan ringkasan yang cukup representatif terhadap isi dokumen, meskipun masih dapat ditingkatkan dengan metode yang lebih kompleks.

Secara keseluruhan, sistem ini layak digunakan sebagai implementasi proyek UAS Sistem Temu Kembali Informasi dan telah memenuhi aspek pencarian, pengelompokan dokumen, peringkasan teks, serta evaluasi performa sesuai dengan ketentuan yang diberikan.